In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from string import punctuation
import csv
import pandas

np.random.seed(1)

Using TensorFlow backend.


In [2]:
df = pandas.read_csv("data/train.csv", index_col="id")
df.head()

D:\Tools\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
id,,,,,,,,,,,,,,,,,,,,,
59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [3]:
nparray = df.values
X = nparray[:,1]   #comment_text column
Y = nparray[:, 0]   #target column

In [4]:
print("X shape - ", X.shape)
print("Y shape - ", Y.shape)

X shape -  (1804874,)
Y shape -  (1804874,)


In [5]:
X

array(["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
       "Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",
       'This is such an urgent design problem; kudos to you for taking it on. Very impressive!',
       ..., 'thank you ,,,right or wrong,,, i am following your advice',
       'Anyone who is quoted as having the following exchange, even if apocryphal, would have received my vote! \n\nBessie Braddock: "Winston, you are drunk, and what’s more you are disgustingly drunk."\nWinston Churchill: "Bessie, my dear, you are ugly, and what’s more, you are disgustingly ugly. But tomorrow I shall be sober and you will still be disgustingly ugly."',
       'Students defined as EBD are legally just as disabled and eligible for special services as a developmentally disabled or physically disabled student. \n\nEMOTIONAL AND BEHAVIORAL DISORDER (EBD).\nDefinition. \nAn 

In [6]:
Y

array([0.0, 0.0, 0.0, ..., 0.0, 0.6212121212121212, 0.0], dtype=object)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.01)

In [8]:
print("Train data:")
print("X_train shape - ", X_train.shape)
print("Y_train shape - ", Y_train.shape)
print("Test data")
print("X_test shape - ", X_test.shape)
print("Y_test shape - ", Y_test.shape)

Train data:
X_train shape -  (1786825,)
Y_train shape -  (1786825,)
Test data
X_test shape -  (18049,)
Y_test shape -  (18049,)


In [9]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words, words_to_index, index_to_words, word_to_vec_map

In [10]:
vocabulary, word_to_index, index_to_word, word_to_vec_map = read_glove_vecs("data/glove.6B.50d.txt")

In [11]:
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


In [12]:
print("length of word_to_index - ", len(word_to_index))
print("length of word_to_vec_map - ", len(word_to_vec_map))

length of word_to_index -  400000
length of word_to_vec_map -  400000


In [13]:
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [14]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence = strip_punctuation(X[i])
        sentence_words =sentence.lower().split()
        filtered_words = [word for word in sentence_words if word in vocabulary]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in filtered_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            #Truncate the sentences with maxlength
            if j == max_len - 1:
                break;
            # Increment j to j + 1
            j = j+1
            
    
    return X_indices

In [15]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [16]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [17]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


In [18]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    #X = LSTM(128, return_sequences=True)(c)
    # Add dropout with a probability of 0.5
    #X = Dropout(rate=0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    #X = LSTM(128)(X)
    X = LSTM(128, activation='sigmoid')(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(rate=0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(1, activation='sigmoid')(X)
    # Add a softmax activation
    X = Activation('sigmoid')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=[sentence_indices], outputs=[X])
    
    
    return model

In [19]:
#maxLen = len(max(X_train, key=len).split())

#use 90 percentile length instead of maxlength to reduce number of steps. For this problem maxlen = 317 90thpercentile = 128
length_list = [len(x.split()) for x in X_train]
maxLen = np.percentile(length_list, 90)
maxLen = int(maxLen)
#print(np.percentile(length_list, 90))

model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 128, 50)           20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 20,091,827
Trainable params: 91,777
Non-trainable params: 20,000,050
___________________________________________________________

In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)

In [22]:
X_train_indices[0:5]

array([[357266., 166369., 216139., 237534., 329916., 188287., 269798.,
        357266., 146233., 287877., 114253., 334274., 269798., 288009.,
        360915., 129404.,  43010., 162321., 268046., 371755., 216139.,
        383772., 269798.,  43010., 115975.,  69709.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.,
      

In [23]:
# Train the neural network only for a small sample - DUBUG PURPOSES

X_train_indices = X_train_indices[0:10000]
Y_train = Y_train[0:10000]
model.fit(X_train_indices, Y_train, epochs = 3, batch_size = 100, shuffle=True)

Epoch 1/3
10000/10000 [==============================] - 26s 3ms/step - loss: 0.7161 - acc: 0.0021
Epoch 2/3
10000/10000 [==============================] - 24s 2ms/step - loss: 0.6935 - acc: 0.0021
Epoch 3/3
10000/10000 [==============================] - 23s 2ms/step - loss: 0.6934 - acc: 0.0021


In [24]:
train_loss, train_accuracy = model.evaluate(X_train_indices, Y_train)

10000/10000 [==============================] - 7s 723us/step


In [25]:
print("Train Accuracy = " , train_accuracy)
print("Train Loss = " , train_loss)
#model.save_weights("lstm_model_1.h5")

Train Accuracy =  0.0021
Train Loss =  0.693267527961731


In [26]:
X_test = X_test[0:1000]
Y_test = Y_test[0:1000]
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
loss, acc = model.evaluate(X_test_indices, Y_test)
print()
print("Test accuracy = ", acc)

1000/1000 [==============================] - 1s 789us/step

Test accuracy =  0.002


In [27]:
Y_test_pred = model.predict(X_test_indices)

In [28]:
import csv

with open('pred_output_2layer_lstm_0427.csv', mode='w') as pred_output:
    output_writer = csv.writer(pred_output, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    for i in range(Y_test.shape[0]):
        output_writer.writerow([X_test[i].encode("utf-8"), Y_test[i], Y_test_pred[i]])

In [82]:
import h5py
f = h5py.File('lstm_model_1.h5', 'r')
print(list(f.keys()))


['dense_5', 'dropout_6', 'embedding_8', 'input_6', 'lstm_7']


In [83]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [112]:
weights = model.layers[2].get_weights()

In [118]:
len(weights)
weights[2].shape

(512,)

In [111]:
weights[0]

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.58402 ,  0.39031 ,  0.65282 , ..., -1.2338  ,  0.46715 ,
         0.78858 ],
       [-1.0588  ,  0.26952 ,  0.94632 , ...,  0.31138 ,  0.79843 ,
         0.20392 ],
       ...,
       [ 0.32301 ,  0.32305 , -0.3136  , ...,  1.8267  , -0.43051 ,
        -1.1958  ],
       [ 0.38715 ,  0.098414, -0.73857 , ...,  1.0611  , -0.2125  ,
        -0.65514 ],
       [-0.019789, -0.040119,  0.86891 , ...,  0.23708 , -0.18194 ,
         0.24766 ]], dtype=float32)